# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [26]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

from newsapi import NewsApiClient

In [27]:
load_dotenv()

True

In [28]:
# Read your api key environment variable
# YOUR CODE HERE!
api_key = os.getenv("NEWS_API_KEY")

In [29]:
type(api_key)

str

In [30]:
type(os.getenv("NEWS_API_KEY"))

str

In [31]:
# Create a newsapi client
# YOUR CODE HERE!
newsapi = NewsApiClient(api_key=api_key)

In [32]:
# Fetch the Bitcoin news articles
# YOUR CODE HERE!
bitcoin_articles = newsapi.get_everything(q="Bitcoin AND bitcoin", language="en", page_size=100, sort_by='relevancy')
print(f"Total Number of Articles with Bitcoin: {bitcoin_articles['totalResults']}")
bitcoin_articles['articles'][0]
type(bitcoin_articles)

Total Number of Articles with Bitcoin: 3712


dict

In [33]:
# Fetch the Ethereum news articles
# YOUR CODE HERE!
ethereum_articles = newsapi.get_everything(q="Ethereum AND ethereum", language="en", page_size=100, sort_by='relevancy')
print(f"Total Number of Articles with Ethereum: {bitcoin_articles['totalResults']}")
type(ethereum_articles)

Total Number of Articles with Ethereum: 3712


dict

In [34]:
print(f"Total Number of Articles with Ethereum: {ethereum_articles['totalResults']}")
ethereum_articles['articles'][0]

Total Number of Articles with Ethereum: 1398


{'source': {'id': 'mashable', 'name': 'Mashable'},
 'author': 'Stan Schroeder',
 'title': 'Crypto wallet MetaMask finally launches on iOS and Android, and it supports Apple Pay',
 'description': "If you've interacted with cryptocurrencies in the past couple of years, there's a good chance you've used MetaMask. It's a cryptocurrency wallet in the form of a browser extension that supports Ethereum and its ecosystem, making it easy to connect with a dece…",
 'url': 'https://mashable.com/article/metamask-ios-android/',
 'urlToImage': 'https://mondrian.mashable.com/2020%252F09%252F02%252Ffd%252Fe724b5edb4b644dba45958e17ad591e1.6b9c6.png%252F1200x630.png?signature=xIKBM112GVhTA9mUq0DRjCVGWSE=',
 'publishedAt': '2020-09-02T16:00:00Z',
 'content': "If you've interacted with cryptocurrencies in the past couple of years, there's a good chance you've used MetaMask. It's a cryptocurrency wallet in the form of a browser extension that supports Ether… [+2291 chars]"}

In [17]:
print(f"articles with Ethereum: {ethereum_articles['totalResults']}")
ethereum_articles["articles"][0]

articles with Ethereum: 1398


{'source': {'id': 'mashable', 'name': 'Mashable'},
 'author': 'Stan Schroeder',
 'title': 'Crypto wallet MetaMask finally launches on iOS and Android, and it supports Apple Pay',
 'description': "If you've interacted with cryptocurrencies in the past couple of years, there's a good chance you've used MetaMask. It's a cryptocurrency wallet in the form of a browser extension that supports Ethereum and its ecosystem, making it easy to connect with a dece…",
 'url': 'https://mashable.com/article/metamask-ios-android/',
 'urlToImage': 'https://mondrian.mashable.com/2020%252F09%252F02%252Ffd%252Fe724b5edb4b644dba45958e17ad591e1.6b9c6.png%252F1200x630.png?signature=xIKBM112GVhTA9mUq0DRjCVGWSE=',
 'publishedAt': '2020-09-02T16:00:00Z',
 'content': "If you've interacted with cryptocurrencies in the past couple of years, there's a good chance you've used MetaMask. It's a cryptocurrency wallet in the form of a browser extension that supports Ether… [+2291 chars]"}

In [18]:
# trying to make function that in turn makes dataframes from api
def make_df(news):
    articles = []
    for item in news:
        try:
            text = item['content']
            articles.append({'text': text})
        except AttributeError:
            pass
    return pd.DataFrame(articles)
        

In [19]:
# trying to make dataframe for each topic
bitcoin_df = make_df(bitcoin_articles['articles'])
bitcoin_df.dropna(inplace=True)
bitcoin_df.head()

,text
0,Two alleged crypto traders in Singapore appare...
1,"By Alexis Akwagyiram, Tom Wilson\r\n* Monthly ..."
2,“The COVID-19 pandemic has resulted in a mass ...
3,"LAGOS/LONDON (Reuters) - Four months ago, Abol..."
4,"LAGOS/LONDON (Reuters) - Four months ago, Abol..."


In [20]:
ethereum_df = make_df(ethereum_articles['articles'])
ethereum_df.dropna(inplace=True)
ethereum_df.head()

,text
0,If you've interacted with cryptocurrencies in ...
1,TL;DR: The Complete Stock and Cryptocurrency I...
2,"September\r\n18, 2020\r\n6 min read\r\nOpinion..."
4,LONDON (Reuters) - It sounds like a surefire b...
5,NEW YORK (Reuters) - Brooklyn-based technology...


In [21]:
# Create the Bitcoin sentiment scores DataFrame
# YOUR CODE HERE!
#trying to make sentiment scores dictionaries
text_snt = {'text_compound': [], 'text_pos': [], 'text_neu': [], 'text_neg': []}

for index, row in bitcoin_df.iterrows():
    try:
        text_sentiment = analyzer.polarity_scores(row['text'])
        text_snt['text_compound'].append(text_sentiment['compound'])
        text_snt['text_pos'].append(text_sentiment['pos'])
        text_snt['text_neu'].append(text_sentiment['neu'])
        text_snt['text_neg'].append(text_sentiment['neg'])
        
        #bitcoin_sentiments.append({"text": text, "date": date, "compound": compound, "positive": pos, "negative": neg, "neutral": neu})
        
    except AttributeError:
        pass
    
bitcoin_sentiment_df = pd.DataFrame(text_snt)

bitcoin_df = bitcoin_df.join(bitcoin_sentiment_df)

bitcoin_df.columns = ['Article_Name', 'Compound', 'Good', 'Neutral', 'Bad']

bitcoin_df.head()

,Article_Name,Compound,Good,Neutral,Bad
0,Two alleged crypto traders in Singapore appare...,-0.6908,0.000,0.840,0.16
1,"By Alexis Akwagyiram, Tom Wilson\r\n* Monthly ...",0.0000,0.000,1.000,0.00
2,“The COVID-19 pandemic has resulted in a mass ...,0.2732,0.063,0.937,0.00
3,"LAGOS/LONDON (Reuters) - Four months ago, Abol...",0.0000,0.000,1.000,0.00
4,"LAGOS/LONDON (Reuters) - Four months ago, Abol...",0.0000,0.000,1.000,0.00


In [22]:
bitcoin_df.head()

,Article_Name,Compound,Good,Neutral,Bad
0,Two alleged crypto traders in Singapore appare...,-0.6908,0.000,0.840,0.16
1,"By Alexis Akwagyiram, Tom Wilson\r\n* Monthly ...",0.0000,0.000,1.000,0.00
2,“The COVID-19 pandemic has resulted in a mass ...,0.2732,0.063,0.937,0.00
3,"LAGOS/LONDON (Reuters) - Four months ago, Abol...",0.0000,0.000,1.000,0.00
4,"LAGOS/LONDON (Reuters) - Four months ago, Abol...",0.0000,0.000,1.000,0.00


In [23]:
bitcoin_df.shape

(94, 5)

In [24]:
bitcoin_df.describe()

,Compound,Good,Neutral,Bad
count,88.000000,88.000000,88.000000,88.000000
mean,0.157626,0.075557,0.882318,0.042114
std,0.464106,0.062797,0.084259,0.068476
min,-0.865800,0.000000,0.588000,0.000000
25%,0.000000,0.000000,0.831750,0.000000
50%,0.272300,0.085000,0.910000,0.000000
75%,0.507000,0.091000,0.924000,0.085000
max,0.923100,0.314000,1.000000,0.294000


In [25]:
# Create the ethereum sentiment scores DataFrame
# YOUR CODE HERE!
text_snt = {'text_compound': [], 'text_pos': [], 'text_neu': [], 'text_neg': []}

for index, row in ethereum_df.iterrows():
    try:
        text_sentiment = analyzer.polarity_scores(row['text'])
        text_snt['text_compound'].append(text_sentiment['compound'])
        text_snt['text_pos'].append(text_sentiment['pos'])
        text_snt['text_neu'].append(text_sentiment['neu'])
        text_snt['text_neg'].append(text_sentiment['neg'])
        
        #bitcoin_sentiments.append({"text": text, "date": date, "compound": compound, "positive": pos, "negative": neg, "neutral": neu})
        
    except AttributeError:
        pass
    
ethereum_sentiment_df = pd.DataFrame(text_snt)

ethereum_df = ethereum_df.join(ethereum_sentiment_df)

ethereum_df.columns = ['Article_Name', 'Compound', 'Good', 'Neutral', 'Bad']

ethereum_df.head()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Frank\anaconda3\envs\pyvizenv\lib\site-packages\IPython\core\interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-3fc77cc5bf8b>", line 7, in <module>
    text_sentiment = analyzer.polarity_scores(row['text'])
  File "C:\Users\Frank\anaconda3\envs\pyvizenv\lib\site-packages\nltk\sentiment\vader.py", line 361, in polarity_scores
    self.constants.REGEX_REMOVE_PUNCTUATION)
  File "C:\Users\Frank\anaconda3\envs\pyvizenv\lib\site-packages\nltk\sentiment\vader.py", line 274, in __init__
    self.words_and_emoticons = self._words_and_emoticons()
  File "C:\Users\Frank\anaconda3\envs\pyvizenv\lib\site-packages\nltk\sentiment\vader.py", line 306, in _words_and_emoticons
    words_punc_dict = self._words_plus_punc()
  File "C:\Users\Frank\anaconda3\envs\pyvizenv\lib\site-packages\nltk\sentiment\vader.py", line 293, in _words_plus_punc
    punc_before = {"".join(p): p[1] fo

KeyboardInterrupt: 

In [ ]:
ethereum_df.head()

In [ ]:
ethereum_df.describe()

In [ ]:
# Describe the Bitcoin Sentiment
# YOUR CODE HERE!
bitcoin_df.head()

In [ ]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!
ethereum_df.head()

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
print(punctuation)

In [ ]:
# Expand the default stopwords list if necessary
# YOUR CODE HERE!
### Student Author wishes not to have too many stopwords...

In [ ]:
#bitk_df = bitcoin_df['text']
#bitk_df

In [ ]:
#period = bitk_df[0].split('.')[0]
#period

In [ ]:
#comma = period.split(',')[0]
#print(comma.split(' '))

In [ ]:
#print(word_tokenize(comma))

In [ ]:
#word_tkod = []
#for clump in sentence_tkod:
#    list_of_words = []
#    for comma in clump:
#        list_of_words = list_of_words + word_tokenize(comma)
#    word_tkod.append(list_of_words)

In [ ]:
#content = bitcoin_articles['articles'][1]['content']
#words = word_tokenize(content)
#content

In [ ]:
#bitcoin_df['tokens'] = bitcoin_df['text'].apply(tokenizer)

In [ ]:
#bitcoin_df.head()

In [ ]:
# Complete the tokenizer function


frank_lemmatizer = WordNetLemmatizer()
def tokenizer(text):
    words = []
    # Cut Punctuation
    regex_application = re.compile("[^a-zA-Z ]")
    repunctuated = regex_application.sub('', text)
    # Make List of Words
    unit = sent_tokenize(repunctuated)
    words = []
    for item in unit:
        subunit = word_tokenize(item)
        words = words + subunit
    # Change Words to Lowercase
    lower_case_words = [subunit.lower() for subunit in words]
    #Cut Stop Words
    stoppers = set(stopwords.words('english'))
    words_stoppers = [subunit for subunit in lower_case_words if subunit not in stoppers]
    # Lemmatize into Root Words
    results = [frank_lemmatizer.lemmatize(subunit) for subunit in words_stoppers]
    return results




In [ ]:
# Create a new tokens column for bitcoin
# YOUR CODE HERE!
bitcoin_df['Tokens'] = [tokenizer(text) for text in bitcoin_df['Article_Name']]
bitcoin_df.head()

In [ ]:
# Create a new tokens column for ethereum
# YOUR CODE HERE!
ethereum_df['Tokens'] = [tokenizer(text) for text in ethereum_df['Article_Name']]
ethereum_df.head()

---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!
large_string = ' '.join(bitcoin_df['Article_Name'])
finished = tokenizer(large_string)
bigram = ngrams(finished, n = 2)
Counter(bigram).most_common(10)

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!
et_large_string = ' '.join(ethereum_df['Article_Name'])
et_finished = tokenizer(et_large_string)
et_bigram = ngrams(et_finished, n = 2)
Counter(et_bigram).most_common(10)

In [ ]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Get the top 10 words for Bitcoin
# YOUR CODE HERE!

large_string = ' '.join(bitcoin_df['Article_Name'])
finished = tokenizer(large_string)
token_count(finished)

In [ ]:
# Get the top 10 words for Ethereum
# YOUR CODE HERE!
et_large_string = ' '.join(ethereum_df['Article_Name'])
et_finished = tokenizer(et_large_string)
token_count(et_finished)

# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!
large_string = ' '.join(bitcoin_df['Article_Name'])
finished = tokenizer(large_string)
entering_text = ' '.join(finished)
word_cloud_bitcoin = WordCloud().generate(entering_text)
plt.imshow(word_cloud_bitcoin)

#bigram = ngrams(finished, n = 2)
#Counter(bigram).most_common(10)

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

et_large_string = ' '.join(ethereum_df['Article_Name'])
et_finished = tokenizer(et_large_string)
et_entering_text = ' '.join(et_finished)
word_cloud_ethereum = WordCloud().generate(et_entering_text)
plt.imshow(word_cloud_ethereum)

#et_large_string = ' '.join(ethereum_df['Article_Name'])
#et_finished = tokenizer(et_large_string)
#et_bigram = ngrams(et_finished, n = 2)
#Counter(et_bigram).most_common(10)

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Optional - download a language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [ ]:
# Concatenate all of the bitcoin text together
# YOUR CODE HERE!
bitcoin_text = ' '.join(bitcoin_df['Article_Name'])
bitcoin_text

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!
bitcoin_document = nlp(bitcoin_text)
# Add a title to the document
# YOUR CODE HERE!
bitcoin_document.user_data["title"] = "Named Entity Recognition (NER) for Bitcoin"

In [ ]:
# Render the visualization
# YOUR CODE HERE!
displacy.render(bitcoin_document, style='ent')

In [ ]:
# List all Entities
# YOUR CODE HERE!
for entity in bitcoin_document.ents:
    print(entity.text, entity.label_)




---

## Ethereum NER

In [ ]:
# Concatenate all of the bitcoin text together
# YOUR CODE HERE!
ethereum_text = ' '.join(ethereum_df['Article_Name'])
ethereum_text

#bitcoin_text = ' '.join(bitcoin_df['Article_Name'])
#bitcoin_text

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!


ethereum_document = nlp(ethereum_text)
# Add a title to the document
# YOUR CODE HERE!
ethereum_document.user_data["title"] = "Named Entity Recognition (NER) for Ethereum"



#bitcoin_document = nlp(bitcoin_text)
# Add a title to the document
# YOUR CODE HERE!
#bitcoin_document.user_data["title"] = "Named Entity Recognition (NER) for Bitcoin"

In [ ]:
# Render the visualization
# YOUR CODE HERE!
# Render the visualization
# YOUR CODE HERE!
displacy.render(ethereum_document, style='ent')

In [ ]:
# List all Entities
# YOUR CODE HERE!
for entity in ethereum_document.ents:
    print(entity.text, entity.label_)